In [1]:
%load_ext autoreload
%autoreload 2
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
from site_health.users.models import User
from website.models import Site
from scraper.models import *
from metric.models import *
from scraper.tasks import *
user = User.objects.get(username='simon')
# page = Page.objects.all().last()
site = Site.objects.get_or_create(url='https://www.simonnouwens.nl', owner=user)[0]
site = Site.objects.get_or_create(url='https://www.ok.nl', owner=user)[0]

In [56]:
scraper = HeadersScraper().factory(page=site.page_set.all()[0])

In [34]:
scraper.get_data()

<Response [301]>

In [61]:
scraper.get_data().headers

{'Server': 'nginx', 'Date': 'Wed, 24 Jun 2020 14:00:26 GMT', 'Content-Type': 'text/html; charset=UTF-8', 'Connection': 'keep-alive', 'X-Powered-By': 'PHP/7.3.19, PleskLin', 'Expires': 'Wed, 24 Jun 2020 15:00:26 GMT', 'Cache-Control': 'max-age=3600', 'X-Redirect-By': 'WordPress', 'Vary': 'Accept-Encoding,User-Agent', 'Location': 'https://ok.nl/', 'Access-Control-Allow-Origin': '*, *', 'Referrer-Policy': 'no-referrer-when-downgrade'}

In [41]:
scraper = GetScraper().factory(page=site.page_set.all()[0])

In [67]:
scraper.__dict__

{'_state': <django.db.models.base.ModelState at 0x7f80cdbb3f10>,
 'id': UUID('293a90dc-7e2c-4f4f-a245-e93d4b099fa0'),
 'page_id': 150,
 'scraped_at': datetime.datetime(2020, 6, 24, 14, 0, 25, 109704, tzinfo=<UTC>),
 'status': 1}

In [3]:
RedirectMetrics().factory(site.page_set.all()[0]).run()

In [2]:
scraper = SeleniumScraper().factory(site.page_set.all()[0])
scraper.wait_ms_before_screenshot = 500
scraper.take_screenshot=True
scraper.save()

In [3]:
selenium_scraper_execute.delay(scraper.id)

INFO 2020-06-24 14:53:55,773 trace 139254 140219026143040 Task scraper.tasks.selenium_scraper_execute[14a38f76-53f5-46d4-a943-33fccee74b1a] succeeded in 14.4049404230027s: None


<EagerResult: 14a38f76-53f5-46d4-a943-33fccee74b1a>

In [31]:
from celery.task.control import inspect

In [42]:
Page.objects.get_or_create(url_part='', site=Site.objects.all()[0])

(<Page: https://simonnouwens.nl>, True)